# Memento, Composite y Flyweight para un Programa de Editor Gráfico

Este notebook implementa un editor gráfico que utiliza los siguientes **tres patrones de diseño**:

1. **Memento**: para guardar/restaurar el estado del editor.
2. **Composite**: para tratar objetos individuales y grupos de objetos de forma uniforme.
3. **Flyweight**: para compartir datos entre múltiples objetos similares y reducir el uso de memoria.


## Patrón: Memento

**Definición**: Permite capturar y guardar el estado interno de un objeto sin violar su encapsulamiento, de modo que pueda restaurarse posteriormente.

**Ventajas**:
- Se pueden deshacer cambios fácilmente.
- No se rompe el encapsulamiento del objeto original.

**Desventajas**:
- Puede consumir mucha memoria si el estado es grande o hay muchos guardados.

**En este ejemplo**: El editor puede guardar su lista de formas antes de un cambio, y restaurarla si se desea deshacer.

In [24]:
import copy

class EditorMemento:
    def __init__(self, state):
        # Se guarda una copia profunda del estado (lista de formas)
        self._state = copy.deepcopy(state)

    def get_state(self):
        return self._state

class History:
    def __init__(self):
        self._states = []

    def push(self, memento):
        self._states.append(memento)

    def pop(self):
        return self._states.pop() if self._states else None

## Patrón: Composite

**Definición**: Permite componer objetos en estructuras de árbol para representar jerarquías parte-todo. Trata objetos individuales y compuestos de forma uniforme.

**Ventajas**:
- Simplifica el código del cliente que usa estructuras jerárquicas.
- Se puede tratar un grupo como si fuera un solo objeto.

**Desventajas**:
- Puede hacer más difícil restringir tipos específicos de componentes.

**En este ejemplo**: Un `Group` de formas puede contener círculos, rectángulos u otros grupos.
El editor puede manejar cualquier `Shape` (individual o compuesta) con una sola interfaz.

In [25]:
from abc import ABC, abstractmethod

# Interfaz base común
class Shape(ABC):
    @abstractmethod
    def draw(self):
        pass

# Clase para círculos
class Circle(Shape):
    def __init__(self, x: int, y: int, radius: int, color: str):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color  # Este atributo será compartido (Flyweight)

    def draw(self):
        print(f"Dibuja Círculo en ({self.x}, {self.y}), radio {self.radius}, color {self.color}")

# Grupo de formas (puede contener otras formas o grupos)
class Group(Shape):
    def __init__(self):
        self.children: list[Shape] = []

    def add(self, shape: Shape):
        self.children.append(shape)

    def draw(self):
        print("Dibuja Grupo:")
        for child in self.children:
            child.draw()

## Patrón: Flyweight

**Definición**: Permite compartir objetos entre instancias para reducir el uso de memoria. Los datos compartidos (intrínsecos) se reutilizan entre objetos similares.

**Ventajas**:
- Ahorro de memoria significativo al compartir datos comunes.
- Ideal para manejar miles de objetos similares (como en un editor gráfico).

**Desventajas**:
- Puede ser más difícil de depurar y entender.
- Introduce complejidad al separar estado intrínseco y extrínseco.

**En este ejemplo**: Usamos una fábrica (`ShapeColorFactory`) para compartir colores entre círculos que usan el mismo color.

In [26]:
class ShapeColorFactory:
    def __init__(self):
        # Diccionario para compartir colores entre formas
        self._shared_colors: dict[str, str] = {}

    def get_color(self, color: str) -> str:
        if color not in self._shared_colors:
            self._shared_colors[color] = color  # En la práctica podría ser un objeto más complejo
        return self._shared_colors[color]

## Clase Editor que integra los tres patrones

- Usa **Memento** para guardar/restaurar el estado.
- Maneja **Shape** y **Group** gracias a Composite.
- Usa **ShapeColorFactory** para reutilizar colores (Flyweight).

In [27]:
class Editor:
    def __init__(self):
        self.shapes: list[Shape] = []
        self.factory = ShapeColorFactory()

    def add_circle(self, x: int, y: int, radius: int, color: str):
        shared_color = self.factory.get_color(color)
        self.shapes.append(Circle(x, y, radius, shared_color))

    def add_shape(self, shape: Shape):
        self.shapes.append(shape)

    def draw(self):
        print("Editor actual:")
        for shape in self.shapes:
            shape.draw()

    def create_memento(self):
        return EditorMemento(self.shapes)

    def restore(self, memento):
        self.shapes = memento.get_state()

In [28]:
# Ejemplo de uso del editor con los tres patrones
editor = Editor()
history = History()

# Agregamos formas individuales
editor.add_circle(10, 10, 5, "red")
editor.add_circle(20, 20, 10, "red")  # Reutiliza el mismo color (Flyweight)

# Guardamos estado (Memento)
history.push(editor.create_memento())

# Agregamos un grupo
group = Group()
group.add(Circle(1, 2, 3, "blue"))
group.add(Circle(4, 5, 6, "blue"))
editor.add_shape(group)

# Dibujamos
editor.draw()

# Deshacer
editor.restore(history.pop())
print("\nDespués de deshacer:")
editor.draw()

Editor actual:
Dibuja Círculo en (10, 10), radio 5, color red
Dibuja Círculo en (20, 20), radio 10, color red
Dibuja Grupo:
Dibuja Círculo en (1, 2), radio 3, color blue
Dibuja Círculo en (4, 5), radio 6, color blue

Después de deshacer:
Editor actual:
Dibuja Círculo en (10, 10), radio 5, color red
Dibuja Círculo en (20, 20), radio 10, color red
